In [ ]:
# Importações para projetos práticos
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Web frameworks
import streamlit as st
import gradio as gr
from flask import Flask, request, jsonify
from fastapi import FastAPI, HTTPException
import uvicorn

# ML e NLP
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

# Deployment e monitoramento
import joblib
import pickle
import requests
import json
from datetime import datetime
import logging

# Visualização avançada
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import altair as alt

# Utilities
import os
import time
import sqlite3
from pathlib import Path

# NLP
import nltk
from textblob import TextBlob
import spacy

# Datasets e utils
import sys
sys.path.append('..')
from datasets.textos_exemplo import *
from utils.nlp_utils import *

# Configurar logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("🚀 Ambiente para projetos práticos configurado!")
print("🔧 Frameworks web: Streamlit, Gradio, FastAPI, Flask")
print("📊 Visualizações: Plotly, Altair")
print("🤖 ML: Transformers, scikit-learn")
print("🎯 Pronto para desenvolver projetos end-to-end!")


In [ ]:
# Projeto 1: Sistema de Análise de Sentimentos
print("🚀 PROJETO 1: SISTEMA DE ANÁLISE DE SENTIMENTOS")
print("=" * 60)

# 1. Preparar e treinar modelo
print("\n📊 1. Preparando dados e treinando modelo...")

# Dataset expandido
reviews_data = {
    'texto': [
        # Positivos
        "Excelente produto, recomendo muito!",
        "Adorei a qualidade, superou expectativas.",
        "Muito bom, chegou rápido e bem embalado.",
        "Produto incrível, vale cada centavo!",
        "Estou muito satisfeito com a compra.",
        "Qualidade excepcional, comprarei novamente.",
        "Fantástico! Melhor que imaginava.",
        "Produto maravilhoso, entrega rápida.",
        "Simplesmente perfeito, recomendo a todos.",
        "Excelente custo-benefício, muito bom.",
        "Adorei o produto, muito útil no dia a dia.",
        "Superou todas as expectativas, incrível!",
        "Produto de alta qualidade, vale a pena.",
        "Muito satisfeito, chegou antes do prazo.",
        "Excelente atendimento e produto top!",
        
        # Negativos
        "Produto terrível, não recomendo.",
        "Muito ruim, dinheiro jogado fora.",
        "Qualidade péssima, chegou quebrado.",
        "Horrível, não serve para nada.",
        "Decepcionante, esperava muito mais.",
        "Produto de baixa qualidade, muito caro.",
        "Não gostei, vou devolver.",
        "Péssimo atendimento e produto ruim.",
        "Não vale o preço, muito caro.",
        "Produto defeituoso, não funcionou.",
        "Muito ruim, não recomendo a ninguém.",
        "Péssima qualidade, chegou danificado.",
        "Produto horrível, perda de tempo.",
        "Não gostei nada, muito decepcionante.",
        "Qualidade inferior, não vale o dinheiro."
    ],
    'sentimento': [1]*15 + [0]*15  # 1=positivo, 0=negativo
}

df = pd.DataFrame(reviews_data)
print(f"✅ Dataset criado: {len(df)} exemplos")
print(f"   Positivos: {sum(df['sentimento'])}")
print(f"   Negativos: {len(df) - sum(df['sentimento'])}")

# Treinar modelo
X_train, X_test, y_train, y_test = train_test_split(
    df['texto'], df['sentimento'], test_size=0.2, random_state=42, stratify=df['sentimento']
)

# Pipeline de ML
pipeline_ml = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=1000, ngram_range=(1, 2))),
    ('classifier', VotingClassifier([
        ('nb', MultinomialNB()),
        ('lr', LogisticRegression(random_state=42)),
        ('svm', SVC(probability=True, random_state=42))
    ]))
])

# Treinar
pipeline_ml.fit(X_train, y_train)

# Avaliar
y_pred = pipeline_ml.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"✅ Modelo treinado - Acurácia: {accuracy:.3f}")

# 2. Criar classe do sistema
print("\n🏗 2. Criando sistema de análise...")

class SistemaAnalisesentimentos:
    def __init__(self, modelo):
        self.modelo = modelo
        self.historico = []
        
    def analisar_texto(self, texto):
        """Analisa sentimento de um texto"""
        # Predição
        pred = self.modelo.predict([texto])[0]
        proba = self.modelo.predict_proba([texto])[0]
        
        # Resultado
        sentimento = "Positivo" if pred == 1 else "Negativo"
        confianca = max(proba)
        
        # Salvar no histórico
        resultado = {
            'texto': texto,
            'sentimento': sentimento,
            'confianca': confianca,
            'timestamp': datetime.now()
        }
        self.historico.append(resultado)
        
        return resultado
    
    def analisar_lote(self, textos):
        """Analisa múltiplos textos"""
        resultados = []
        for texto in textos:
            resultado = self.analisar_texto(texto)
            resultados.append(resultado)
        return resultados
    
    def obter_estatisticas(self):
        """Retorna estatísticas do histórico"""
        if not self.historico:
            return {"total": 0}
            
        total = len(self.historico)
        positivos = sum(1 for r in self.historico if r['sentimento'] == 'Positivo')
        negativos = total - positivos
        confianca_media = np.mean([r['confianca'] for r in self.historico])
        
        return {
            'total': total,
            'positivos': positivos,
            'negativos': negativos,
            'percentual_positivo': positivos/total*100,
            'confianca_media': confianca_media
        }
    
    def salvar_modelo(self, caminho):
        """Salva o modelo treinado"""
        joblib.dump(self.modelo, caminho)
        print(f"✅ Modelo salvo em: {caminho}")
    
    def carregar_modelo(self, caminho):
        """Carrega modelo salvo"""
        self.modelo = joblib.load(caminho)
        print(f"✅ Modelo carregado de: {caminho}")

# Instanciar sistema
sistema = SistemaAnalisesentimentos(pipeline_ml)

# 3. Testar sistema
print("\n🧪 3. Testando sistema...")

textos_teste = [
    "Este produto é fantástico, recomendo!",
    "Muito ruim, não gostei nada.",
    "Qualidade média, poderia ser melhor.",
    "Excelente compra, muito satisfeito!",
    "Produto ok, nada demais.",
    "Simplesmente horrível, péssimo!"
]

print("📝 Análises individuais:")
for texto in textos_teste:
    resultado = sistema.analisar_texto(texto)
    print(f"'{texto}'")
    print(f"  → {resultado['sentimento']} (confiança: {resultado['confianca']:.3f})")
    print()

# Estatísticas
stats = sistema.obter_estatisticas()
print("📊 Estatísticas do sistema:")
for key, value in stats.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.3f}")
    else:
        print(f"  {key}: {value}")

# 4. Criar interface simples com Streamlit (código para arquivo separado)
print("\n🖥 4. Código para interface Streamlit:")
print("-" * 40)

streamlit_code = '''
import streamlit as st
import pandas as pd
import plotly.express as px
from datetime import datetime
import joblib

# Configurar página
st.set_page_config(
    page_title="Análise de Sentimentos",
    page_icon="😊",
    layout="wide"
)

# Título
st.title("🎯 Sistema de Análise de Sentimentos")
st.markdown("Analise o sentimento de textos em tempo real!")

# Sidebar
st.sidebar.header("⚙️ Configurações")

# Carregar modelo (simulado)
@st.cache_resource
def carregar_modelo():
    # Aqui você carregaria seu modelo salvo
    # return joblib.load('modelo_sentimentos.pkl')
    return None

# Interface principal
col1, col2 = st.columns([2, 1])

with col1:
    st.header("📝 Análise de Texto")
    
    # Input de texto
    texto_input = st.text_area(
        "Digite o texto para análise:",
        height=100,
        placeholder="Ex: Adorei este produto, muito bom!"
    )
    
    # Botão de análise
    if st.button("🔍 Analisar Sentimento", type="primary"):
        if texto_input:
            # Aqui você faria a predição real
            # resultado = sistema.analisar_texto(texto_input)
            
            # Simulação
            import random
            sentimento = random.choice(["Positivo", "Negativo"])
            confianca = random.uniform(0.7, 0.95)
            
            # Mostrar resultado
            if sentimento == "Positivo":
                st.success(f"😊 Sentimento: {sentimento}")
            else:
                st.error(f"😞 Sentimento: {sentimento}")
            
            st.info(f"🎯 Confiança: {confianca:.1%}")
        else:
            st.warning("⚠️ Digite um texto para análise!")

with col2:
    st.header("📊 Estatísticas")
    
    # Métricas simuladas
    st.metric("Total de Análises", "1,234")
    st.metric("Sentimentos Positivos", "67%", "5%")
    st.metric("Confiança Média", "89%", "2%")

# Seção de análise em lote
st.header("📋 Análise em Lote")

uploaded_file = st.file_uploader(
    "Faça upload de um arquivo CSV com textos:",
    type=['csv']
)

if uploaded_file:
    df = pd.read_csv(uploaded_file)
    st.dataframe(df.head())
    
    if st.button("🚀 Analisar Lote"):
        st.success("✅ Análise em lote concluída!")

# Histórico
st.header("📈 Histórico de Análises")

# Gráfico simulado
dados_historico = pd.DataFrame({
    'Data': pd.date_range('2024-01-01', periods=30, freq='D'),
    'Positivos': np.random.randint(10, 50, 30),
    'Negativos': np.random.randint(5, 30, 30)
})

fig = px.line(
    dados_historico.melt(id_vars='Data', var_name='Sentimento', value_name='Quantidade'),
    x='Data', y='Quantidade', color='Sentimento',
    title="Evolução dos Sentimentos ao Longo do Tempo"
)

st.plotly_chart(fig, use_container_width=True)
'''

print(streamlit_code)

# 5. Criar API com FastAPI (código para arquivo separado)
print("\n🌐 5. Código para API FastAPI:")
print("-" * 40)

fastapi_code = '''
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List
import joblib
import uvicorn

# Criar app
app = FastAPI(
    title="API de Análise de Sentimentos",
    description="API para análise de sentimentos em textos",
    version="1.0.0"
)

# Modelos Pydantic
class TextoInput(BaseModel):
    texto: str

class TextosInput(BaseModel):
    textos: List[str]

class ResultadoSentimento(BaseModel):
    texto: str
    sentimento: str
    confianca: float

# Carregar modelo na inicialização
@app.on_event("startup")
async def carregar_modelo():
    global modelo
    # modelo = joblib.load('modelo_sentimentos.pkl')
    modelo = None  # Simulação

# Endpoints
@app.get("/")
async def root():
    return {"message": "API de Análise de Sentimentos ativa!"}

@app.post("/analisar", response_model=ResultadoSentimento)
async def analisar_sentimento(input_data: TextoInput):
    try:
        # Aqui você faria a predição real
        # resultado = sistema.analisar_texto(input_data.texto)
        
        # Simulação
        import random
        sentimento = random.choice(["Positivo", "Negativo"])
        confianca = random.uniform(0.7, 0.95)
        
        return ResultadoSentimento(
            texto=input_data.texto,
            sentimento=sentimento,
            confianca=confianca
        )
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/analisar-lote", response_model=List[ResultadoSentimento])
async def analisar_lote(input_data: TextosInput):
    try:
        resultados = []
        for texto in input_data.textos:
            # Simulação
            import random
            sentimento = random.choice(["Positivo", "Negativo"])
            confianca = random.uniform(0.7, 0.95)
            
            resultados.append(ResultadoSentimento(
                texto=texto,
                sentimento=sentimento,
                confianca=confianca
            ))
        
        return resultados
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/health")
async def health_check():
    return {"status": "healthy", "timestamp": datetime.now()}

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)
'''

print(fastapi_code)

print("\n✅ PROJETO COMPLETO!")
print("📁 Arquivos para criar:")
print("  - app_streamlit.py (interface web)")
print("  - api_fastapi.py (API REST)")
print("  - modelo_sentimentos.pkl (modelo salvo)")
print("  - requirements.txt (dependências)")
print("  - Dockerfile (containerização)")

# Salvar modelo para uso posterior
sistema.salvar_modelo('modelo_sentimentos.pkl')

print(f"\n🎯 Sistema pronto para produção!")
print(f"📊 Acurácia do modelo: {accuracy:.3f}")
print(f"🔧 Total de análises realizadas: {len(sistema.historico)}")
print(f"📈 Estatísticas: {sistema.obter_estatisticas()}")
